# 线性系统求解
Ax = b,$$A = c_0A_0+c_1A_1+c_2A_2=\mathbb{I}+0.2X_0Z_1+0.2X_0$$
$$|b\rangle = U|0\rangle = H_0H_1H_2|0\rangle$$


In [9]:
import numpy as np
import matplotlib
import paddle 
from paddle import matmul
from paddle_quantum.circuit import UAnsatz
from paddle_quantum.utils import random_pauli_str_generator, pauli_str_to_matrix, dagger
#参数设置
num_qubit = 3  
num_shots = 10**6 # Number of quantum measurements
tot_qubits = num_qubit+1 # Addition of an ancillary qubit
ancilla_idx = num_qubit #Index of the ancillary qubit (last position)
ITR = 80 #训练迭代次数
LR = 0.5 #学习速率
seed = 0 # Seed for random number generator
#构造cz gate
a= np.array([np.pi/2])
cz_theta = paddle.to_tensor(a)
#coefficient of A=c_0 A_0 + c_1 A_1 ...
c = np.array([1,0.2,0.2])




In [10]:
#定义门电路U和A
def U_b():
    for idx in range(num_qubit):
        cir.h(idx)
def CA(idx):
    if idx == 0:
        # Identity operation
        None
    elif idx == 1:
        cir.cnot([ancilla_idx,0])
        #cz gate
        cir.s(ancilla_idx)
        cir.s(ancilla_idx)
        cir.s(ancilla_idx)
        cir.cnot([ancilla_idx,1])
        cir.rz(cz_theta,1)
        cir.cnot([ancilla_idx,1])
        cir.rz(-cz_theta,1)
    elif idx == 2:
        cir.cnot([ancilla_idx, 0])
#定义变分量子电路 使得|x> = V|0>
def variational_block(theta):
    '''
    QNN
    '''
    #第一层是给除附加位外所有量子比特施加h门
    for idx in range(num_qubit):
        cir.h(idx)
    #单层的全y门
    for idx, element in enumerate(theta):
        cir.ry(element,idx)


In [11]:
#hadamard test
def local_hadamard_test(theta,l = None,lp=None,j = None,part = None):
    #初始化电路
    cir = UAnsatz(tot_qubits)#需要附加位
    #作用在附加位上的H门
    cir.h(ancilla_idx)
    #如果要计算的是虚数部，附加位还需要加一个相位门
    #phase gate
    if part == 'Im' or part == 'im':
        cir.s(ancilla_idx)
    #生成一个向量｜x>
    variational_block(theta)
    #使用A_l
    CA(l)
    #添加U_b
    U_b()
    #添加受控z
    if j != -1:
        #cz gate
        cir.s(ancilla_idx)
        cir.s(ancilla_idx)
        cir.s(ancilla_idx)
        cir.cnot([ancilla_idx,j])
        cir.rz(cz_theta,j)
        cir.cnot([ancilla_idx,j])
        cir.rz(-cz_theta,j)
    #添加U
    U_b()
    #contralled gate为A的伴随矩阵
    CA(lp)
    #辅助位第二个hadamard门
    cir.h(ancilla_idx)
    #对辅助比特为｜0>进行观测
    fi_state = cir.run_state_vector()
    state = paddle.reshape(fi_state, shape=(2**tot_qubits, 1))
    # print(state.shape)
    M_0 = np.array([[1,0],[0,0]])
    Id = np.identity(2)
    M = np.kron(np.kron(Id,Id),Id)
    M = np.kron(M,M_0)
    M = paddle.to_tensor(M)
    #计算辅助位｜0>的概率
    M = paddle.matmul(dagger(M), M)
    p0 = paddle.matmul(paddle.matmul(dagger(state), M), state) 
    p0 = p0.numpy().real
    return p0*2 - 1


    

In [12]:
#计算local cost function的系数
def mu(theta,l = None,lp=None,j=None):
    mu_real = local_hadamard_test(theta,l = l,lp = lp,j = j,part='Re')
    mu_imag = local_hadamard_test(theta,l = l, lp = lp, j = j, part= 'Im')
    return mu_real + mu_imag * 1.0j

In [13]:
# 归一化 先计算 <x|A'A|x>
def psi_norm(theta):
    '''Returns the normalization constant <psi|psi>, where |psi> = A |x>.'''
    norm = 0
    for l in range(len(c)):
        for lp in range(len(c)):
            norm = norm+c[l]*np.conj(c[lp])*mu(theta,l,lp,-1)
            # norm = norm + c[l]*np.conj(c[lp])
    return abs(norm)
#计算cost function
def local_cost(theta):
    #当A｜x>和|b>同方向时，该值越接近0
    mu_sum = 0
    #
    for l in range(0, len(c)):
        for lp in range(0, len(c)):
            for j in range(0, num_qubit):
                mu_sum = mu_sum + c[l] * np.conj(c[lp]) * mu(theta, l, lp, j)

    mu_sum = abs(mu_sum)
    m1 = num_qubit * psi_norm(theta)
    result = 0.5 - 0.5 * mu_sum / m1
    # Cost function C_L
    return result

In [14]:
theta = np.random.uniform(0,2*np.pi,num_qubit)
print(theta)
theta = paddle.to_tensor(theta)
loss =local_cost(theta)
# print('%.10f' %loss)

[0.54841453 3.62582236 0.84280363]


NameError: name 'cir' is not defined

In [430]:
class vqlsOpt(paddle.nn.Layer):
        # 初始化一个长度为theta_size的可学习参数列表，并用[0, 2*pi]的均匀分布来填充初始值
    # def __init__(self):    
        # self.theta = np.random.uniform(0,2*np.pi,num_qubit)
    def __init__(self, shape, dtype='float64'):
        super(vqlsOpt, self).__init__()
        # 初始化 theta 参数列表，并用 [0, 2*pi] 的均匀分布来填充初始值
        self.theta = self.create_parameter(shape=shape,default_initializer=paddle.nn.initializer.Uniform(low=0.0, high=2*np.pi), dtype=dtype, is_bias=False)
    # 定义损失函数和前向传播机制
    def forward(self):
        #计算损失函数
        loss = local_cost(theta)
        return loss

In [440]:
# 定义网络维度
vqls = vqlsOpt(shape = [1])
# 一般来说，我们利用Adam优化器来获得相对好的收敛，当然你可以改成SGD或者是RMS prop.
opt = paddle.optimizer.Adam(learning_rate = LR, parameters = vqls.parameters())    
# 优化循环
for itr in range(1,ITR+1):
    # 前向传播计算损失函数
    loss = vqls()
    # 反向传播极小化损失函数
    loss.backward()
    opt.minimize(loss)
    opt.clear_grad()
    # 记录学习曲线
    loss_list.append(loss.numpy())
    if itr % 10 == 0:
        print('iter:', itr, '  loss: %.4f' % loss.numpy())


/usr/local/anaconda3/envs/paddle_quantum_env/lib/python3.7/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: 'vqlsOpt' object has no attribute 'create_parameter'

In [443]:
Id = np.identity(2)
Z = np.array([[1, 0], [0, -1]])
X = np.array([[0, 1], [1, 0]])

A_0 = np.identity(8)
A_1 = np.kron(np.kron(X, Z), Id)
A_2 = np.kron(np.kron(X, Id), Id)

A_num = c[0] * A_0 + c[1] * A_1 + c[2] * A_2
b = np.ones(8) / np.sqrt(8)
print("A = \n", A_num)
print("b = \n", b)
A_inv = np.linalg.inv(A_num)
x = np.dot(A_inv, b)
c_probs = (x / np.linalg.norm(x)) ** 2
print(x)

A = 
 [[1.  0.  0.  0.  0.4 0.  0.  0. ]
 [0.  1.  0.  0.  0.  0.4 0.  0. ]
 [0.  0.  1.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  1.  0.  0.  0.  0. ]
 [0.4 0.  0.  0.  1.  0.  0.  0. ]
 [0.  0.4 0.  0.  0.  1.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  1.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  1. ]]
b = 
 [0.35355339 0.35355339 0.35355339 0.35355339 0.35355339 0.35355339
 0.35355339 0.35355339]
[0.25253814 0.25253814 0.35355339 0.35355339 0.25253814 0.25253814
 0.35355339 0.35355339]


In [444]:
#验证
import paddle
from paddle_quantum.circuit import UAnsatz
cir = UAnsatz(num_qubit)
def prepare_x(theta):
    variational_block(theta)
    

